<a href="https://colab.research.google.com/github/SeongwonTak/TIL_swtak/blob/master/Pytorch/DL_with_pytorch_ch3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pytorch로 시작하는 딥러닝 3장

추천 시스템 개발 연습에 미뤄두었던 Pytorch로 시작하는 딥러닝을 다시 읽기 시작했다.

## 신경망의 구성 요소
딥러닝 알고리즘 학습을 위해서는 다음것들이 필요하다.
- Data Pipeline의 구축
- 신경망 아키텍처의 구축
- 오차함수를 이용한 아키텍처의 평가
- 최적화 알고리즘을 통한 가중치 최적화

여기서, 데이터를 입력받고 변환, 데이터 출력 등의 진행이 일어나는 것이 Layer이다.

### Linear Layer
입력 데이터에 선형 변환을 실시한다.

In [ ]:
from torch.nn import Linear
myLayer = Linear(in_features=10, out_features=5, bias=True)

In [ ]:
import torch
inputs = torch.randn(10)
myLayer(inputs)

tensor([ 0.3043,  0.3279, -0.5284,  0.1655,  0.1435], grad_fn=<AddBackward0>)

여기서 weight과 bias를 확인할 수 있다.

In [ ]:
myLayer.weight

Parameter containing:
tensor([[ 0.0607, -0.0157, -0.1399, -0.2390,  0.3099,  0.1081,  0.3100,  0.0038,
          0.1653, -0.1457],
        [-0.0105, -0.1517,  0.1091, -0.3012,  0.0748,  0.1228,  0.2021, -0.3095,
          0.1208,  0.0443],
        [-0.0350, -0.3157, -0.0953,  0.0062,  0.0467, -0.2658, -0.1403, -0.0814,
         -0.2917, -0.0564],
        [-0.1532, -0.1766, -0.0888, -0.1542,  0.1929,  0.2613, -0.2622,  0.0666,
         -0.0416,  0.0662],
        [ 0.1608,  0.2548, -0.1372, -0.2340,  0.1273,  0.2292, -0.2390, -0.0089,
          0.1897, -0.1561]], requires_grad=True)

In [ ]:
myLayer.bias

Parameter containing:
tensor([-0.0602, -0.1389, -0.1575,  0.1735, -0.1273], requires_grad=True)

복수 레이어 또한 구현이 가능하며, 이의 경우는 다음 레이어에 이전 레이어의 출력을 입력하면 된다.

In [ ]:
#복수 레이어의 예시
myLayer1 = Linear(10, 5)
myLayer2 = Linear(5, 2)
myLayer2(myLayer1(inputs))

tensor([-0.0125,  0.5872], grad_fn=<AddBackward0>)

### 활성화 함수
물론, 당연히 이미 밑딥애서 배웠지만, 선형함수로만 층을 쌓는것은 의미가 없다.
Pytorch에서 활성화 함수의 구현은 다음과 같다.

In [ ]:
# ReLU와 Sigmoid의 사용법
from torch.nn import ReLU
from torch.nn import Sigmoid

sample_data = torch.Tensor([[1, 2, -1, -0.5, 3]])
relu = ReLU()  # 이렇게 함수를 한 번 정의하고 넘어가야 한다...
sigmoid = Sigmoid()  # 이것도.
print(relu(sample_data))
print(sigmoid(sample_data))

tensor([[1., 2., 0., 0., 3.]])
tensor([[0.7311, 0.8808, 0.2689, 0.3775, 0.9526]])


## 딥러닝 알고리즘의 구현
(어찌보면 당연한거지만) 신경망은 class 형태로 개발된다.

In [ ]:

class MyFirstNewtork(nn.Module):
  def __init__(self, input_size, hidden_size, output_size):
    super(MyFirstNewtork, self).__init__()
    self.layer1 = nn.Linear(input_size, hidden_size)
    self.layer2 = nn.Linear(hidden_size,output_size)

  def forward(self, input):
    out = self.layer1(input)
    out = nn.ReLU(out)
    out = self.layer2(out)
    return out

NameError: ignored

### 마지막 레이어의 결정
해결하려는 문제의 유형에 따라, 마지막 레이어의 형태가 결정된다.
- 회귀 : 선형 레이어의 적용하여 연속적인 값 1개 출력
- 이진분류 : 시그모이드를 활용하여 0 또는 1에 가까운 값 출력
- 다중분류 : 소프트맥스 함수를 활용하여, 개별 확률을 반환

### 오차함수

In [ ]:
# 회귀함수의 경우
loss = nn.MSELoss()
input = Variable(torch.randn(3,5), requires_grad = True)
target = Variable(torch.randan(3,5))
output = loss(input, target)
output.backward()

In [ ]:
# 분류를 위한 엔트로피 구현 - torch ver
loss = nn.CrossEntropyLoss()
input = Variable(torch.randn(3, 5), requires_grad = True)
target = variable(torch.LongTensor(3).random_(5))
output = loss(input, target)
output.backward()


In [ ]:
# 교차 엔트로피 구현
def cross_entropy(true_label, prediction):
  if true_label == 1:
    return -log(prediction)
  else:
    return -log(1 - prediction)

### 가중치의 최적화

In [ ]:
optimizer = optim.SGD(model.parameters(), lr = 0.01)
for input, target in dataset:
  optimizer.zero_grad()
  output = model(input)
  loss = loss_fn(output, target)
  loss.backward()
  optimizer.step()

## 딥러닝을 이용한 이미지 분류
다음과 같은 단계로 이미지 분류를 실시한다.
1. 데이터 분류 실시
2. 학습과 검증 데이터 셋의 분류
3. Tensor로 데이터 로딩
  - 이미지 같은 크기로 저장
  - 데이터셋 정규화
  - 이미지 데이터셋을 Pytorch Tensor로 변환
4. 배치처리 형태로 Pytorch Tensor로 로딩
5. 네트워크 아키텍처 구축
6. 모델 학습

세부적인 내용은 앞으로 배울 내용이므로, 각 부분별로 어떻게 구성되어있는지 개요만 부분적으로 확인해보자.

In [ ]:
# 데이터 분류 실시

In [ ]:
# 학습과 검증 데이터 셋 분류

In [ ]:
# Tensor로 데이터 로딩
simple_transform = transforms.Compose([transforms.Scle((224,224)),
                                       transforms.ToTensor(),
                                       transforms.Normalize(
                                           [0.485, 0.456, 0.406],
                                           [0.229, 0.224, 0.225]
                                       )])
train = ImageFolder('dogsandcats/train/',simple_transform)
valid = ImageFolder('dogsandcats/valid/',simple_transform)

여기서 train_class는
- train.class_to_idx - cat 0, dog 1
- train.classes - [cat, dog]

In [ ]:
# 배치처리 형태로 Tensor 로딩
train_data_gen = torch.utils.data.DataLoder(train,batch_size = 64, num_workers = 3)
valid_data_gen = torch.utils.data.DataLoder(valid,batch_size = 64, num_workers = 3)

네트워크 아키텍처는 컴퓨터 비전의 경우는 ImageNet, ResNet을 사용할 것이다.

In [ ]:
model_ft = models.resnet18(pretained = True)  # 인스턴스 출력
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, 2)
if is_cuda:
  model_ft = model_ft.cuda()

In [ ]:
# 모델의 학습 - 오차함수와 옵티마이저
# 엔트로피로 오차함수, SDG를 기반으로 옵티마이저.
# StepLR은 학습률의 조정.
learning_rate = 0.001
criterion = nn.CrossEntropyLoss()
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum = 0.9)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size = 7, gamma = 0.1)

In [ ]:
# 최종_train_model
def train_model(model, criterion, optimizer, scheduler, num_epochs = 25):
  since = time.time()

  best_model_wts = model.state_dict()
  best_acc = 0.0

  for epoch in range(num_epochs):
    print('Epoch {}/{}'.format(epoch, num_epochs-1))
    print('-' * 10)

    # 각 에폭의 구성 : 학습, 검증
    for phase in ['train', 'valid']:
      if phase == 'train':
        scheduler.step()
        model.train(True)
        model.train(False)

        running_loss = 0.0
        running_corrects = 0

        for data in dataloaders[phase]:

In [ ]:
even_sum = 0
for i in range(100):
  if i % 2 == 0:
    even_sum += i
print(even_sum)

2450
